<a href="https://colab.research.google.com/github/greyhound101/gan_segmentation_FE/blob/main/gan_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive
Mounted at /content/gdrive


In [2]:
%%writefile util.py
import tensorflow as tf
import os
import glob
def string_tensor_from_idlist_and_path(type, name = None):
  file_names = glob.glob('/content/gdrive/MyDrive/2dsegmentation/'+type+'_*.npy')
  string_tensor = tf.Variable(file_names, trainable=False,
                              name=name, validate_shape=False)
  return string_tensor

Writing util.py


In [ ]:
!pip install -U tensorflow==1.13.0rc1

     |████████████████████████████████| 92.7MB 34kB/s 
     |████████████████████████████████| 3.1MB 39.8MB/s 
     |████████████████████████████████| 368kB 49.9MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 


In [ ]:
!pip install -U scipy==1.2.0

In [ ]:
image_file_image.shape

In [ ]:
%%writefile load_folder_images.py
import tensorflow as tf
import numpy as np
import glob
import random
def _decode_and_preprocess_image(image_file, shift_param = -128, rescale_param = 128, resized_image_size = [128, 128]):
  
  image = tf.cast(image_file, tf.float32)
  
  if shift_param != 0:
    image = tf.add(image, shift_param)
  
  if rescale_param != 0:
    image = tf.multiply(image, 1.0/rescale_param)
  
  
  
  #image = tf.random_crop(image, [32, 32, 3])
  print(image.shape)
  image = tf.image.resize_images(image, resized_image_size) 
  print(image.shape)
  image.set_shape((resized_image_size[0], resized_image_size[1], 1))
  #image = tf.image.grayscasle_to_rgb(image, 'torgb')
  return image


def _load_images(image_file, batch_size, num_preprocess_threads, min_queue_examples, shift_param = -128, rescale_param = 128, resized_image_size = [128, 128], shuffle = True):
  
  image = _decode_and_preprocess_image(image_file, shift_param, rescale_param, resized_image_size)
  #image = tf.image.grayscasle_to_rgb(image, 'torgb')
  images = []
  if shuffle == True:
    images = tf.train.shuffle_batch(
          [image],
          batch_size=batch_size,
          num_threads=num_preprocess_threads,
          capacity=min_queue_examples + 3 * batch_size,
          min_after_dequeue=min_queue_examples)
  else:
    images = tf.train.batch(
          [image],
          batch_size=batch_size,
          num_threads=num_preprocess_threads,
          capacity=min_queue_examples + 3 * batch_size)
  
  
  return images

def load_image_and_segmentation_from_idlist(idlist_tensor_img, idlist_tensor_seg, batch_size, num_preprocess_threads, min_queue_examples, shift_params = [-128, -0.5], rescale_params = [128, 0.5], resized_image_size = [128, 128], shuffle = True):

  filename_queue = random.sample(glob.glob('/content/gdrive/MyDrive/2dsegmentation/'+'image'+'_*.npy'),4)
  image_file_image=[]
  for path in filename_queue:
    image_file_image.append(np.expand_dims(np.resize(np.load(path),(128,128)),-1))
  image_file_image=np.stack(image_file_image,0)
  image_file_image[image_file_image>250]=250
  image_file_image[image_file_image<-200]=-200
  image_file_image=(image_file_image-np.mean(image_file_image))/np.std(image_file_image)
  processed_image=tf.convert_to_tensor(image_file_image)
  processed_image = tf.cast(processed_image, tf.float32)


  filename_queue = random.sample(glob.glob('/content/gdrive/MyDrive/2dsegmentation/'+'segmentation'+'_*.npy'),4)
  image_file_seg=[]
  for path in filename_queue:
    image_file_seg.append(np.expand_dims(np.resize(np.load(path),(128,128)),-1))
  image_file_seg=np.stack(image_file_seg,0)
  processed_seg=tf.convert_to_tensor(image_file_seg)
  processed_seg = tf.cast(processed_seg, tf.float32)
  
  # if shuffle == True:
  #   images, segmentations = tf.train.shuffle_batch(
  #         [processed_image, processed_seg],
  #         batch_size=batch_size,
  #         num_threads=num_preprocess_threads,
  #         capacity=min_queue_examples + 3 * batch_size,
  #         min_after_dequeue=min_queue_examples)
  # else:
  #   images, segmentations = tf.train.batch(
  #         [processed_image, processed_seg],
  #         batch_size=batch_size,
  #         num_threads=num_preprocess_threads,
  #         capacity=min_queue_examples + 3 * batch_size)
    
  return processed_image, processed_seg
  

def load_images_from_idlist(idlist, batch_size, num_preprocess_threads, min_queue_examples, shift_param = -128, rescale_param = 128, resized_image_size = [128, 128], shuffle = True):
  # Make a queue of file names including all the image files in the relative
  # image directory.
  filename_queue = tf.train.string_input_producer(idlist,
                                                  shuffle=shuffle)
  
  # Read an entire image file. If the images
  # are too large they could be split in advance to smaller files or use the Fixed
  # reader to split up the file.
  image_reader = tf.WholeFileReader()
  
  # Read a whole file from the queue, the first returned value in the tuple is the
  # filename which we are ignoring.
  _, image_file = image_reader.read(filename_queue)

  return _load_images(image_file, batch_size, num_preprocess_threads, min_queue_examples, shift_param, rescale_param, resized_image_size, shuffle)

def load_images(folder_path_match, batch_size, num_preprocess_threads, min_queue_examples, shift_param = -128, rescale_param = 128, resized_image_size = [128, 128], shuffle = True):
  # Make a queue of file names including all the image files in the relative
  # image directory.
  filename_queue = tf.train.string_input_producer(
    tf.train.match_filenames_once(folder_path_match),
    shuffle=shuffle)
  
  # Read an entire image file. If the images
  # are too large they could be split in advance to smaller files or use the Fixed
  # reader to split up the file.
  image_reader = tf.WholeFileReader()
  
  # Read a whole file from the queue, the first returned value in the tuple is the
  # filename which we are ignoring.
  _, image_file = image_reader.read(filename_queue)

  return _load_images(image_file, batch_size, num_preprocess_threads, min_queue_examples, shift_param, rescale_param, resized_image_size, shuffle)

def load_image(image_path, num_preprocess_threads, min_queue_examples, resized_image_size = [64, 64]):
  # Make a queue of file names including all the image files in the relative
  # image directory.
  filename_queue = tf.train.string_input_producer(
    tf.train.match_filenames_once(image_path))
  
  # Read an entire image file. If the images
  # are too large they could be split in advance to smaller files or use the Fixed
  # reader to split up the file.
  image_reader = tf.WholeFileReader()
  
  # Read a whole file from the queue, the first returned value in the tuple is the
  # filename which we are ignoring.
  _, image_file = image_reader.read(filename_queue)
  
  image = tf.image.decode_png(image_file);
  
  #image.set_shape((32, 32, 3))
  
  
  image = tf.cast(image, tf.float32)
  
  image = tf.add(image, -128)
  image = tf.multiply(image, 1.0/128)
  
  
  
  #image = tf.random_crop(image, [32, 32, 3])
  image = tf.image.resize_images(image, resized_image_size) 
  image.set_shape((resized_image_size[0], resized_image_size[1], 1))
  #image = tf.image.grayscasle_to_rgb(image, 'torgb')
  
  images = tf.train.shuffle_batch(
      [image],
      batch_size=1,
      num_threads=num_preprocess_threads,
      capacity=min_queue_examples + 3 * 1,
      min_after_dequeue=min_queue_examples)
  
  
  
  return images

In [ ]:
%%writefile dcgan.py
import tensorflow as tf

def linear(input_tensor, output_dim, scope=None, stddev=1.0):
    norm = tf.random_normal_initializer(stddev=stddev)
    const = tf.constant_initializer(0.0)
    with tf.variable_scope(scope or 'linear'):
        w = tf.get_variable('w', [input_tensor.get_shape()[1], output_dim], initializer=norm)
        b = tf.get_variable('b', [output_dim], initializer=const)
        return tf.matmul(input_tensor, w) + b
      
def minibatch(input_tensor, num_kernels=256, kernel_dim=4):
        in_vec = tf.reshape(input_tensor, [input_tensor.get_shape().as_list()[0], -1])
        x = linear(in_vec, num_kernels * kernel_dim, scope='minibatch', stddev=0.02)
        activation = tf.reshape(x, (-1, num_kernels, kernel_dim))
        diffs = tf.expand_dims(activation, 3) - tf.expand_dims(tf.transpose(activation, [1, 2, 0]), 0)
        abs_diffs = tf.reduce_sum(tf.abs(diffs), 2)
        minibatch_features = tf.reduce_sum(tf.exp(-abs_diffs), 2)
        #minibatch_features = tf.reshape(minibatch_features, (128, 4, 4, -1))
        test_out = tf.concat(axis=1, values=[in_vec, minibatch_features])
        test_out = tf.reshape(test_out, [input_tensor.get_shape().as_list()[0], input_tensor.get_shape().as_list()[1], input_tensor.get_shape().as_list()[2], -1])
        return test_out

def disc_conv2d(input_tensor, num_inputs, num_outputs, kernel_size, stride, dropout_ratio, scope):
  with tf.variable_scope(scope):
              w = tf.get_variable(
                        'w',
                        [kernel_size, kernel_size, num_inputs, num_outputs],
                        tf.float32,
                        tf.truncated_normal_initializer(stddev=0.05))
              b = tf.get_variable(
                  'b',
                  [num_outputs],
                  tf.float32,
                  tf.zeros_initializer())
              c = tf.nn.conv2d(input_tensor, w, [1, stride, stride, 1], 'SAME')
              mean, variance = tf.nn.moments(c, [0, 1, 2])
              outputs = leaky_relu(tf.nn.batch_normalization(c, mean, variance, b, None, 1e-5))
              #outputs = tf.nn.dropout(outputs, 0.5)
              return outputs;
              #out.append(outputs)

class Generator:
    def __init__(self, depths=[1024, 512, 256, 128, 64], f_size=4):
        self.reuse = False
        self.f_size = f_size
        self.depths = depths + [2]

    def model(self, inputs):
        i_depth = self.depths[0:(len(self.depths) - 1)]
        o_depth = self.depths[1:(len(self.depths))]
        out = []
        
        with tf.variable_scope('g', reuse=self.reuse):
            # reshape from inputs
            inputs = tf.convert_to_tensor(inputs)
            with tf.variable_scope('fc_reshape'):
                w0 = tf.get_variable(
                    'w',
                    [inputs.get_shape()[-1], i_depth[0] * self.f_size * self.f_size],
                    tf.float32,
                    tf.truncated_normal_initializer(stddev=0.05))
                b0 = tf.get_variable(
                    'b',
                    [i_depth[0]],
                    tf.float32,
                    tf.zeros_initializer())
                fc = tf.matmul(inputs, w0)
                reshaped = tf.reshape(fc, [-1, self.f_size, self.f_size, i_depth[0]])
                mean, variance = tf.nn.moments(reshaped, [0, 1, 2])
                outputs = tf.nn.relu(tf.nn.batch_normalization(reshaped, mean, variance, b0, None, 1e-5))
                out.append(outputs)
            # deconvolution (transpose of convolution) x 4
            
            
            for i in range(len(self.depths) - 1):
                    
                with tf.variable_scope('conv%d' % (i + 1)):
                    kernel_size = 5
    
                     
                    w = tf.get_variable(
                        'w',
                        [kernel_size, kernel_size, o_depth[i], i_depth[i]],
                        tf.float32,
                        tf.truncated_normal_initializer(stddev=0.05))
                    b = tf.get_variable(
                        'b',
                        [o_depth[i]],
                        tf.float32,
                        tf.zeros_initializer())
                    dc = tf.nn.conv2d_transpose(
                        outputs,
                        w,
                        [
                            int(outputs.get_shape()[0]),
                            self.f_size * 2 ** (i + 1),
                            self.f_size * 2 ** (i + 1),
                            o_depth[i]
                        ],
                        [1, 2, 2, 1])
                    if i < len(self.depths) - 2:
                        mean, variance = tf.nn.moments(dc, [0, 1, 2])
                        outputs = tf.nn.relu(tf.nn.batch_normalization(dc, mean, variance, b, None, 1e-5))
                    else:
                        outputs = tf.nn.bias_add(dc, b)
                        outputs = tf.nn.tanh(outputs)
                    out.append(outputs)
                    
        
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return out

    def __call__(self, inputs):
        return self.model(inputs)


class Discriminator:
    def __init__(self, depths=[64, 128, 256, 512]):
        self.reuse = False
        self.depths = [2] + depths
        


    def model(self, inputs, return_feature_vector=False):
        def leaky_relu(x, leak=0.2):
            return tf.maximum(x, x * leak)
        i_depth = self.depths[0:(len(self.depths) - 1)]
        o_depth = self.depths[1:(len(self.depths))]
        out = []
        self.weights = []
        with tf.variable_scope('d', reuse=self.reuse):
            outputs = inputs
            # convolution x 4

            for i in range(len(self.depths) - 1):
                with tf.variable_scope('conv%d' % i):
                    w = tf.get_variable(
                        'w',
                        [5, 5, outputs.get_shape()[3], o_depth[i]],
                        tf.float32,
                        tf.truncated_normal_initializer(stddev=0.05))
                    b = tf.get_variable(
                        'b',
                        [o_depth[i]],
                        tf.float32,
                        tf.zeros_initializer())
                    c = tf.nn.conv2d(outputs, w, [1, 2, 2, 1], 'SAME')
                    mean, variance = tf.nn.moments(c, [0, 1, 2])
                    outputs = leaky_relu(tf.nn.batch_normalization(c, mean, variance, b, None, 1e-5))
                    
                    self.weights.append(w);
                    
                    out.append(outputs)
                    
            if(return_feature_vector == True):
                return out[3]      
            
            with tf.variable_scope('classify'):
                dim = 1
                for d in outputs.get_shape()[1:].as_list():
                    dim *= d
                    print(dim)
                w = tf.get_variable('w', [dim, 2], tf.float32, tf.truncated_normal_initializer(stddev=0.02))
                b = tf.get_variable('b', [2], tf.float32, tf.zeros_initializer())
                out.append(tf.nn.bias_add(tf.matmul(tf.reshape(outputs, [-1, dim]), w), b))
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')
        return out

    def __call__(self, inputs, return_feature_vector=False):
        return self.model(inputs, return_feature_vector)


class DCGAN:
    def __init__(self,
                 batch_size=128, f_size=2, z_dim=400,
                 gdepth1=512, gdepth2=256, gdepth3=128, gdepth4=128,
                 ddepth1=128,   ddepth2=128, ddepth3=256, ddepth4=512):
        self.batch_size = batch_size
        self.f_size = f_size
        self.z_dim = z_dim
        self.g = Generator(depths=[gdepth1, gdepth2, gdepth3, gdepth4, gdepth4, gdepth4], f_size=self.f_size)
        self.d = Discriminator(depths=[ddepth1, ddepth2, ddepth3, ddepth4, ddepth4, ddepth4])
        self.z = tf.random_uniform([self.batch_size, self.z_dim], minval=-1.0, maxval=1.0)
        self.losses = {
            'g': None,
            'd': None
        }

    def build(self, input_images,
              learning_rate=0.0004, beta1=0.5, feature_matching=False):
        """build model, generate losses, train op"""
        generated_images = self.g(self.z)[-1]
        print('before d gen')
        
        outputs_from_g = self.d(generated_images)
        
        print('before d real')
        outputs_from_i = self.d(input_images)
        logits_from_g = outputs_from_g[-1]
        logits_from_i = outputs_from_i[-1]
  
        tf.add_to_collection(
            'g_losses',
            tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                    logits=logits_from_g, labels=tf.ones([self.batch_size], dtype=tf.int64))))
        
        
        d_loss_real = tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                     logits=logits_from_i, labels=tf.ones([self.batch_size], dtype=tf.int64)))
        
        d_loss_fake = tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                    logits=logits_from_g, labels=tf.zeros([self.batch_size], dtype=tf.int64)))
        
        tf.add_to_collection(
             'd_losses', d_loss_real
             )
        tf.add_to_collection(
            'd_losses',d_loss_fake
            )
        
 
        
        if feature_matching:
            features_from_g = tf.reduce_mean(outputs_from_g[-2], axis=(0))
            features_from_i = tf.reduce_mean(outputs_from_i[-2], axis=(0))
            tf.add_to_collection('g_losses', tf.multiply(tf.nn.l2_loss(features_from_g - features_from_i), 0.1))
            mean_image_from_g = tf.reduce_mean(generated_images, axis=(0))
            mean_image_from_i = tf.reduce_mean(input_images, axis=(0))
            tf.add_to_collection('g_losses', tf.multiply(tf.nn.l2_loss(mean_image_from_g - mean_image_from_i), 0.01))

        self.losses['g'] = tf.add_n(tf.get_collection('g_losses'), name='total_g_loss')
        self.losses['d'] = tf.add_n(tf.get_collection('d_losses'), name='total_d_loss')
        g_opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1)
        d_opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1)

        self.g_opt_op = g_opt.minimize(self.losses['g'], var_list=self.g.variables)
        self.d_opt_op = d_opt.minimize(self.losses['d'], var_list=self.d.variables)
        
        with tf.control_dependencies([self.g_opt_op, self.d_opt_op]):
            self.train = tf.no_op(name='train')
        return self.train

    def sample_images(self, row=8, col=8, num_samples = None, inputs=None):
        if inputs is None:
            inputs = self.z
            
        if num_samples is None:
            num_samples = self.batch_size
            
        inputs = tf.random_uniform([num_samples, self.z_dim], minval=-1.0, maxval=1.0)
        images = tf.cast(tf.multiply(tf.add(self.g(inputs)[-1], 1.0), 127.5), tf.uint8)
          
        return images
        images = [image for image in tf.split(axis=0, num_or_size_splits=self.batch_size, value=images)]
        rows = []
        for i in range(row):
            rows.append(tf.concat(axis=2, values=images[col * i + 0:col * i + col]))
        image = tf.concat(axis=1, values=rows)
        return tf.image.encode_jpeg(tf.squeeze(image, [0]))
      
    def retrieve_feature_vector(self, inputs):
        vector = tf.reshape(self.d(inputs, return_feature_vector=True), [-1]) 
        return vector

In [ ]:
%%writefile train.py
from datetime import datetime
import time
import glob
import tensorflow as tf

from dcgan import *
import numpy as np 
from scipy.misc import imsave
import load_folder_images
import os
from load_folder_images import load_image_and_segmentation_from_idlist
from load_folder_images import load_image
import csv
import util

FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('train_dir', '/tmp/dcgan_train',
                           """Directory where to write event logs """
                           """and checkpoint.""")
tf.app.flags.DEFINE_integer('max_steps', 100000,
                            """Number of batches to run.""")
tf.app.flags.DEFINE_boolean('log_device_placement', False,
                            """Whether to log device placement.""")

tf.app.flags.DEFINE_string("working_directory", "working_dir", "")
tf.app.flags.DEFINE_string("checkpoint_dir", "checkpoints", "")




z_dim = 100
batch_size = 4
learning_rate = 0.0004
beta1 = 0.5
monitoring_batches = 200; #number of batches after which some samples are saved
num_monitoring_cycles = 100;




def train():

  idlist_img_name = "list_of_img_ids.txt"
  idlist_seg_name = "list_of_seg_ids.txt"
  img_folder_path = "image_folder_path/"
  seg_folder_path = "segmentation_folder_path/"

  idlist_tensor_img = util.string_tensor_from_idlist_and_path('image')
  idlist_tensor_seg = util.string_tensor_from_idlist_and_path('segmentation')
  
  

  images, segmentation_images = load_image_and_segmentation_from_idlist(idlist_tensor_img, idlist_tensor_seg, batch_size, 16, 2560, shift_params = [-128, -0.5], rescale_params = [128, 0.5], shuffle = True)
  print(images.shape, segmentation_images.shape)
  
  dcgan = DCGAN(batch_size)
  input_images = images
  
  input_plus_segmentation = tf.concat([input_images, segmentation_images], 3)
  print(input_plus_segmentation.shape)
  train_op = dcgan.build(input_plus_segmentation)
  print('here')
  sample_images = dcgan.sample_images()
  print('here')  
  saver = tf.train.Saver(max_to_keep = 50)
  print('here')

  
  with tf.Session() as sess:
      summary_writer = tf.summary.FileWriter(FLAGS.train_dir, graph=sess.graph)
      sess.run(tf.global_variables_initializer())
      
      
      coord = tf.train.Coordinator()
      threads = tf.train.start_queue_runners(sess=sess, coord=coord)
      
      total_iters_max =  num_monitoring_cycles * monitoring_batches
      iters = 0
      total_duration = 0.0
      avg_iter_time = 0.0
      
      for cycle in range(num_monitoring_cycles):
        for batch in range(monitoring_batches):
          start_time = time.time()
          _, g_loss_value, d_loss_value = sess.run([train_op, dcgan.losses['g'], dcgan.losses['d']])
          _, g_loss_value, d_loss_value = sess.run([dcgan.g_opt_op, dcgan.losses['g'], dcgan.losses['d']])        
            
          duration = time.time() - start_time
          total_duration = total_duration + duration
          iters = iters + 1
          avg_iter_time = (avg_iter_time * (iters - 1) + duration) / iters
          #ETA_seconds = (total_iters_max / iters) * total_duration - total_duration
          ETA_seconds = (0.95 * avg_iter_time + 0.05 * duration) * (total_iters_max - iters)
          format_str = 'cycle (%d / %d), batch (%d / %d) loss = (G: %.8f, D: %.8f) (%.3f sec/batch) (ETA: %d seconds)'
          print(format_str % (cycle, num_monitoring_cycles, batch, monitoring_batches, g_loss_value, d_loss_value, duration, int(ETA_seconds)))
        
        
        checkpoint_folder = FLAGS.working_directory + "/" + FLAGS.checkpoint_dir + '/checkpoint%d.ckpt' % cycle
        if not os.path.exists(checkpoint_folder):
          os.makedirs(checkpoint_folder)
        saver.save(sess, checkpoint_folder)


        imgs = sess.run(sample_images)
         
        for k in range(batch_size):
            imgs_folder = os.path.join(FLAGS.working_directory, 'out/imgs%d/') % cycle
            if not os.path.exists(imgs_folder):
              os.makedirs(imgs_folder)
              
            segs_folder = os.path.join(FLAGS.working_directory, 'out/segs%d/') % cycle
            if not os.path.exists(segs_folder):
              os.makedirs(segs_folder)
              
            img_channel = imgs[k][:, :, 0]
            img_seg = imgs[k][:, :, 1]
            imsave(os.path.join(imgs_folder, 'img_%d.png') % k,
                     img_channel.reshape(128, 128))
            imsave(os.path.join(segs_folder, 'img_%d.png') % k,
                     img_seg.reshape(128, 128))
              
        imgs_in = sess.run(input_plus_segmentation)    
        for k in range(batch_size):
          imgs_folder = os.path.join(FLAGS.working_directory, 'in/imgs%d/') % cycle
          if not os.path.exists(imgs_folder):
            os.makedirs(imgs_folder)
            
          segs_folder = os.path.join(FLAGS.working_directory, 'in/segs%d/') % cycle
          if not os.path.exists(segs_folder):
            os.makedirs(segs_folder)

          img_channel = imgs_in[k][:, :, 0]
          img_seg = imgs_in[k][:, :, 1]
          imsave(os.path.join(imgs_folder, 'img_%d.png') % k,
                     img_channel.reshape(128, 128))
          imsave(os.path.join(segs_folder, 'img_%d.png') % k,
                     img_seg.reshape(128, 128))
              
      coord.request_stop()
      coord.join(threads)
      
     

def main(argv=None):  # pylint: disable=unused-argument

  
  if tf.gfile.Exists(FLAGS.train_dir):
    tf.gfile.DeleteRecursively(FLAGS.train_dir)
  tf.gfile.MakeDirs(FLAGS.train_dir)
  train()


if __name__ == '__main__':
  tf.app.run()

In [ ]:
!python train.py